# Assignment 01 - Pitfalls in Data Mining     
## CSCI E-96

The goal of data mining is to find important relationships in large complex datasets. These dataset typically contain a large number of variables. The **high-dimensional** nature of the data leads to some commonly encountered pitfalls which lead to incorrect inferences.   

In this assignment you will gain a bit of experience with three important concepts in data mining at a small scale. These methods are essential as the scale increases:  

1. **False Discovery Rate Control:** The goal of data mining is to find important relationships in large complex datasets. These dataset typically contain a large number of variables. The **high-dimensional** nature of the data leads to some commonly encountered pitfalls of incorrect inference, **false discovery**. A related problem is cutting off a large-scale analysis when a desired relationship is 'found'. This practice of **p-value mining** often leads to unwarranted inferences. You will apply false discovery rate (FDR) control methods to address this problem.   
2. **Key-Value Pairs:** Large scale data is typically managed using key-value (KV) pairs. The exercises in this assignment give you some experience working with KV pair data management.  
3. **Map and Reduce Processes:** Much of large scale data mining requires use of a split-apply-combine approach. The data is split into manageable chunks, analytic transformations are applied, and the result combined or aggregated. A commonly used class of a split-apply-combine algorithm is MapReduce. 

In order to keep the scope of this assignment manageable, you will use limited versions of KV pair management and MapReduce. Specifically, you will use common Python tools to implement these concepts rather than dedicated large scale analytic platforms. 

## Multiple Hypothesis Tesing

Testing multiple hypothesis in high-dimensional data is problematic. Exhaustively testing all pairwise relationships between variables in a data set is a commonly used, but generally misleading, from of **multiple comparisons**. The chance of finding false significance, using such a **data dredging** approach, can be surprisingly high. 

In this exercise you will perform multiple comparisons on **only 20 identically distributed independent (iid)** variables. Ideally, such tests should not find significant relationships, but the actual result is quite different. 

To get started, execute the code in the cell below to load the required packages. 

In [ ]:
import pandas as pd
import numpy as np
import numpy.random as nr
from scipy.stats import ttest_ind, f_oneway
from itertools import product, combinations
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import comb
import math
import time

pd.set_option("display.max_rows", None, "display.max_columns", None)

In this exercise you will apply a [t-test](https://www.statisticshowto.com/probability-and-statistics/t-test/) to all pairwise combinations of identical Normally distributed variables. In this case, we will create a data set with 20 iid Normal distributions of 1000 samples each. Execute the code in the cell below to find this data and display the mean and variance of each variable.  

In [ ]:
ncolumns = 20
nr.seed(234)
normal_vars = nr.normal(size=(1000,ncolumns))
print('The means of the columns are\n', np.mean(normal_vars, axis = 0))
print('\nThe variances of the columns are\n', np.var(normal_vars, axis = 0))

Notice that means are close to 0.0 and variances are close to 1.0 respectively. As expected, there is not much difference between these variables.

How many of these t-tests will show **significance** at the 0.05 cut-off level? With 20 variables there are 190 unique pairwise combinations. We expect to find a number of falsely significant test results from this many pairwise tests. 

## Creating a hash 

The goal of this exercise is to compute pairwise hypothesis tests of the differences in means for each of the iid Normal vectors. Here you will compute the **key-value**, $[K,V]$, pairs. Each key must represent an index for the two vectors used to compute the test statistic. The keys will then be used to index the results of the hypothesis tests. 

The question is, how can we create a hash from the keys for the pair of vectors? In this case to we will use a simple, but far from optimal hash. For the two vector indicies $i, j$, for some key and modulo, $m$, we will compute the hash as:  

$$h(i,j) = \big(i + key*(j+1) \big) mod\ m$$

> **Exercise 1-1:** Given that our space of vectors is actually quite small, just 20, we do not need a sophisticated and scalable hash function. We take advantage of the fact that Pandas data frames use hash tables for index lookup. This hashed key is used to store and retrieve the values using the indicies of a Pandas data frame, in about $O(N)$ time.     

> In this exercise you will test a simple hash function and its inverse. Examine the code below and notice that the hash function encodes the two indexes into a single integer by simple additional and multiplication. The modulo operation limits the size of the hash table. However, to keep things simple you will not implement any hash collision resolution mechanism. As a result, the size of the table is set much larger than required.  

> To test this hash, do the following:    
> 1. Create a function called `hash_function` to compute the hash using the formula given above. The arguments to the function are $i$ and $j$, the $hash\_key$ and the $modulo\_multiplier$. The defaults of the arguments are $hash\_key=32$ and $m$ the $modulo\_multiplier=32$. The hash is then the product $\big(i + hash\_key*(j+1) \big) modulo\ m$ where $m = hash\_key * modulo\_multiplier$. This approach gives a high ratio of possible keys, $m$ to the number range of values $n$, or the number of variables. 
> 2. Using the Python [ittertools.combinations](https://docs.python.org/3/library/itertools.html#itertools.combinations) function create all unique pairwise combinations of indexes i and j. The arguments to this function are the indexes to the iid Normal vectors. The iterator is `range(ncolumns)` choose 2, since these comparisons are pairwise.    
> 3. Within this loop call the hash with the values of $i$ and $j$ as arguments.   
> 3. On a single line print the following; the values of i and j, the hash key value, but only if $i \le 3$. The restriction is to keep the printed output shorter.   

In [ ]:
## Your code goes below
def hash_function(i, j, hash_key=32, modulo_multiplier=32):
    ## Put your code below. 
   



for i,j in combinations(range(ncolumns), 2):
    if i <= 3: 
        hash = hash_function(i,j)
        print('i = ' + str(i) + '  j = ' + str(j) + '   hash = ' + str(hash))

> Examine the key pairs and the hash values. The question is, are there any hash collisions? This can be done as follows:   
> 5. Compute a list of the hash values for all combinations of $i$ and $j$.   
> 6. Print the length of the list.  
> 7. Print the length of the unique values of the hash. You can find the unique values in a list with the [numpy.unique](https://numpy.org/doc/stable/reference/generated/numpy.unique.html) function. 

In [ ]:
## Put your code below. 




> Examine the results you have printed. Is there any evidence of hash key collisions?     
> The ratio of $m/n$ is deliberately kept high since the simple hash function has no collision resolution mechanism. Optionally, you can try reducing this ration (the multiplier) to 16 and 8, noting the increase in hash collisions.  
> **End of exercise.**

> **Answer:**      

## A MapReduce Example   

In the following steps you will use a simplified MapReduce framework to find and sort every possible pairwise t-test result for the `normal_vars` table. To do so you must find all **combinations** of keys, $\{i,j\}$. The MapReduce processes you will create, are intended to create a table of these unique pairwise key combinations. The t-test is then computed from the values in the `normal_vars` table in the Reduce process. This MapReduce process only creates the required key combinations. The values remain in the `normal_vars` table. This approach simplifies some of the bookkeeping.     

In a bit more details the steps for this algorithm are:   
- The **Map process** builds a hash table with the hash of i,j as key and i,j as values. As has been noted most of this hash table will contain NaNs. The hash table is still relatively small, since the values of the vectors used for the statistical calculations are not stored in the table, only the results. Please don't be confused by the switching roles of i,j as they act as keys and values at different times.   
- The **Shuffle** scans the hash table for non-empty rows (not NaN values for i,j) and builds a table of unique combination of keys i and j.   
- The **Reduce process** uses the keys i,j from the Shuffle process to look up the columns of the numeric random variables in the `normal_vars` table. The resulting table contains the i,j pairs as the keys for the random variable lookup. The t-test is applied to these numeric values. The results are sorted and returned.   

It is clear that at this small scale one could compute the t-test results for the combinations of the random variables directly. The point of the exercises is to get some experience working with key-value pairs and the MapReduce framework.  

### The map process

We are constructing this example a map and a reduce process. The processes are intended to compute the hypothesis test for differences of means between all the pairs of vectors. The first step is the map process, which creates the keys, or values of $i$ and $j$ for these pairs. This is an example of a **similarity join**.    

> **Computational Note:** In this exercise you will create your own simple hash table using a Pandas data frame. The data frame will be indexed by the key value computed by hashing the $[i,j]$ pairs, which gives nearly $O(n)$ performance for lookups. This implementation is only for demonstration purposes only and has several significant limitations compared to a production quality implementation. 
> 1. You will use a fixed length datafrsme. We take advantage of the fact the index lookup in Pandas data frames uses a hash table. In other words, referencing a row (or column) of a Pandas data frame is a key-value lookup using a hash table. 
> 2. You will not need to implement a collision resolution mechanism so a larger table is required, which is not particularly memory efficient. At scale, a sparse representation of the table would be used.     
> 3. You will not implement any table resizing mechanism. Again, a fixed size table is limited in the quantity of data the table can store and also resulting in poor memory utilization.  
> As an alternative, we could use the built hashed $[K,V]$ pairs of a Python dictionary. However, a dictionary only allows one key value and we are working with 2, $\{i,j\}$. You could create a hash and then hash this hash with the built in function. In this care we will take the more direct route just described. If you are not familiar with Python dictionaries you can find a short tutorial [here](https://www.tutorialspoint.com/python_data_structure/python_hash_table.htm), as well as many other places on the web.   

> **Exercise 1-2:** You will now create the code for the map which build a data frame with $i, j$ key pairs indexed by the hash. By the following steps you will create code that represents a map task.  a
> 1. Create a data frame with two columns $i$ and $j$ with number of rows $= hash\_key * modulo\_multiplier $ and set all values to $= numpy.nan$.
> 2. Create a loop over all combinations of the pairs of i and j.   
> 2. Compute the hash key value for the indexes, i and j.  
> 4. Add the $i$ and $j$ values to the row indeed by the hash key.  
> 5. Return the hash table.
> 6. Execute the function to create the hash table.
> 7. Compute and print the length of the hash table. 

In [ ]:
def map_hypothesis(vars, hash_key=1024, modulo_multiplier=32):
    ## Put your code below. 
   






hash_table, nhashes = map_hypothesis(normal_vars)
print('length of the table = ' + str(len(hash_table)))
hash_table.head()

> **End of exercise.**

### The shuffle and reduce task

Now that you have the keys for the pairwise combinations of the vectors it is time to perform the reduce process. The reduce process computes the pair-wise t-statistics and p-values. These statistical values are indexed by the keys of the pair of vectors. This process reduces the full vectors of values down to just two numbers for each pair of vectors. 

> **Exercise 1-3:** You will now create and apply the following code for the reduce process:   
> 1. Create an empty data frame, called  `test_results`, with columns, `i`, `j`, `t_statistic`, and `p_value, and length of the previously created hash table.    
> 2. Iterate over the combinations of possible $\{i,j \}$ tuple.  
> 3. For the $i,j$ pairs look up the variable vector values and use them to compute the t-statistic and p-value with the [scipy.stats.ttest_ind](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) function.
> 4. Assign the values to the row in the data frame indexed by the hash key. Use the `hash_function` previously created, which will work with the Pandas `loc` method.     
> 5. Return the data frame, sorted by p-value in ascending order, using the [Pandas.DataFrame.sort_values](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html).    
> 6. Execute your function and save the returned data frame.   
> 7. Display the first 10 rows of the data frame.  

In [ ]:
def reduce_significance(hash_table, values):  
    ## Your code goes here. 
    ## Create a data framreturn the results of the 
    ## the reduce process. The results are grouped by the first 
    ## index i. 
    test_results = pd.DataFrame(index=hash_table.index, columns=['i','j','t_statistic','p_value'])

    ## We do not need a shuffle here, since we simple want to work with the possible 
    ## combinations of i and j, if i not equal to j.  
    ncolumns = values.shape[1]
    for i,j in combinations(range(ncolumns), 2): 
    
    ## Given the i,j pairs we need to compute the t-statistic and the p-value.   
    ## This is the reduce step, since for each i,j pair there is only 
    ## a t-statistic and a p-value. 
    ## Put your code below. 
        
        
    
    
test_stats = reduce_significance(hash_table, normal_vars)       
test_stats.head(10)

> 9. In the cell below, create a filter for pair test cases which are significant and save these cases in a data frame. 
> 10. Print the number (len) of significant results.
> 11. Print the rows with the significant test results.

In [ ]:
significance_level = 0.05
## Put your code below. 




> Notice the large number of apparently 'significant' test results. Answer the following questions:  
> 1. Is the number of false positive cases approximately what you expected?    
> 2. Examine which of the iid Normal vectors contribute to the false positive results. Are there vectors which contribute multiple times?   
> **End of exercise.**

>  **Answers:**   
> 1.        
> 2.      

### Bonferroni correction  

Several adjustments to the multiple comparisons problem have been proposed. Dunn published a method know as the **Bonferroni correction** in 1961. The Bonferroni correction is a widely used method to reduce the false positive rate of hypothesis tests.  The adjustment is simple:
$$\alpha_b = \frac{\alpha}{m}\\
with\\ 
m =\ number\ of\ groups$$

Can the Bonferroni correction help? Yes, by greatly increasing the confidence level required for a statistically significant result. The problem with the Bonferroni correction is the reduction in power as the  grows smaller. For big data problems with large numbers of groups, this issue can be especially serious. In fact, the Bonferroni correction can often lead to significant numbers of missed discoveries, or Type II errors. 

> **Exercise 1-4:** You will now apply the Bonferroni correction to determining the significance of the p-values of the pair-wise t-tests. Do the following:
> 1. Compute the Bonferroni significance level. 
> 2. Apply this cutoff to the p-values of all of the cases.
> 3. Print the number of false discoveries, or Type I errors. 
> Execute your code and examine the results

In [ ]:
## Your code goes below 
significance_bonferroni = significance_level/190.0
significant_tests = test_stats[test_stats.iloc[:,3] < significance_bonferroni]
len(significant_tests)            

> Does using the  Bonferroni correction reduce false significance tests, or even eliminate them?           

> **Answer:**     

> But, can we detect small effect with Bonferroni correction, as this method significantly reduces power of tests? Execute the code in the cell below, which compares a standard Normal to a Normal with a small mean (effect size), to find out. 

In [ ]:
nr.seed(567)
print(significance_bonferroni)
print(ttest_ind(normal_vars[:,0], nr.normal(loc = 0.01, size=(1000,1))))

> Provide a short answer to these questions:        
> 1. Given the Bonferroni correction and this difference in means, would this result be considered significant or would it be a type II error, and why?    
> 2. What does this result tell you about the downside of the Bonferroni correction as a FDR control method, in terms of finding significant results?      
> **End of exercise.**

> **Answers:**      
> 1.      
> 2.     

## Imporved False Discovery Rate Control Methods 

We have seen the potential pitfalls of multiple hypothesis testing. Further, we have seen that a simple approach to **false discovery rate (FDR) control** is not effective. You will now apply more sophisticated FDR control methods. 

The specific problem we will explore is to determine which genes lead to expression of a certain disease. In this example, there are gene expression data for 97 patients. Some of these patients have ulcerative colitis, a condition believed not to be inheritable, and others have Crohn's disease, which is believed to be genetically inherited.    

One approach to this problem is to perform hypothesis tests on the expression of the genes between patients with the two conditions. Since there are over 10,000 genes there is considerable chance for false discovery. Therefore, careful application of FDR control is required.

To continue with the example, execute the code in the cell below to load the data and print the dimensionality of the data frame. 

In [ ]:
gene_data = pd.read_csv('../data/ColonDiseaseGeneData-Cleaned.csv')
print(gene_data.shape)

There are data from 97 patients (rows) with 10,497 genes (features in columns). A large number of pairwise hypothesis tests are required!     

Execute the code in the cell below to view the first 5 columns of the data frame, which includes the expression of the first 4 genes.  

In [ ]:
 print(gene_data.iloc[:,:5])      

You will apply two FDR control methods to these data to reduce the FDR rate, and compare the results to the Bonferroni method. These methods control the FDR, while not being overly conservative on true positive results, like the Bonferronic correction. 

### Holm's method

The first of these **Holm's method**. Holm's method operates on the ordered set of p-values, $D = \{ p_{(1)}, p_{(2)}, p_{(3)}, \ldots, p_{(n)} \}$. The threshold is dynamic with the order of the P-value. For the $ith$ p-value, $p(i)$ for $N$ pairwise tests is:  

$$p(i) \le Threshold(Holm's) = \frac{\alpha}{N - i + 1}$$

For example: for the 10th ordered p-value with 1,000 total tests (genes) and significance level of 0.05, the cutoff is:   

$$p(10) \le \frac{0.05}{1000 - 10 + 1} = 0.00005045$$

### Map process  

> **Exercise 01-5:** To start the processing of these data you will first create and execute code for a map process. The map process groups the data by the patient's disease into 2 data frames, `ulcerative`, and `crohns`. The keys for each of these key-value pairs are the gene identifier. Notice that one key is all that is needed in this case. Now do the following to create and execute a function, `map_gene` to do the following:   
> 1. Create a logical mask and group the values by `Disease State`.
> 2. Use the mask to divide the original data frame into two data frames.
> 3. Return the transpose of the two data frames, removing the `Disease State` column. The result of this operation should be data frames with gene expressions for each patient in the columns and the gene identifier as the row index. 
> 4. Print the heads of the two data frames.  

In [ ]:
def map_gene(gene_data):  
    ## Your code goes below 
    ## First, separate the columns by disease type  
    mask = gene_data.loc[:,'Disease State'] == 'Ulcerative Colitis (UC)'
    ulcerative = gene_data[mask]
    crohns = gene_data[np.logical_not(mask)]
    return ulcerative.iloc[:,1:].transpose(), crohns.iloc[:,1:].transpose()
    
ulcerative, crohns = map_gene(gene_data)    

> 3. Execute the code in the two cells below to display the heads of these data frames and examine the results. 

In [ ]:
ulcerative.head()

In [ ]:
crohns.head()

### Reduce process 

> **Exercise 01-6:** With the key-value pairs organized by disease state, it is time to create and execute code of a reduce process. The reduce process will compute the pairwise t-statistics and p-values for each gene and return the sorted results. Specifically, your `gene_test` with arguments of the two mapped data frames will do the following:   
> 1. Create an empty results data frame with using the hash key (gene identifier) as an index and columns t_statistics, and p-value.
> 2. Use a `for` loop to iterate over the keys of either data frame.  
> 3. Compute the t-statistic and p-value for the gene (key) for the pairwise tests between the data frames.
> 4. Assign the values to the row in the data frame using the gene identifier as index with the Pandas `loc` method.    
> 5. Sort the results data frame, `inplace`, into ascending order.
> 6. Return the resulting data frame.
> 7. Call the function and print the first 10 rows of the data frame.  

In [ ]:
def gene_test(ulcerative, crohns):  
    test_results = test_results = pd.DataFrame(index = ulcerative.index, columns=['t_statistic','p_value'])
    ## Put your code below. 
    
    
    
    
    
    
gene_statistics = gene_test(ulcerative, crohns)    
gene_statistics.head(10)

### Significance of results 

With the gene data reduced to the t-test statistics, you will now determine the significance of these tests. It is important to understand that scientists believe that expression of a disease, like Corhn's, is only in a small number of genes, less than approximately 100.  

> **Exercise 01-6:** As a first step in understanding the gene expression significance complete and execute the code in the cell below to find the number of 'significant' genes using the simple single hypothesis test cutoff criteria.  

In [ ]:
significance_level =0.05
## Put your code below. 




> Does this large number of 'statistically significant' results appear credible, given that only a few genes are thought to have significant expression for this disease and why in terms of FDR?    
> **End of exercise.**     

> **Answer:**      

> **Exercise 01-7:** We have already seen that the Bonferroni correction is a rather conservative approach to testing the significance of large numbers of hypotheses. You will now use the Bonferroni correction to test the significance of the gene expression, by completing the code in the cell below. 

In [ ]:
## Put your code below. 





> Does the foregoing result seems reasonable in terms of discovery rate and FDR control?    
> **End of exercise.**   

> **Answer:**       

> **Exercise 01-08:** It may well be the case that the foregoing results using the Bonferroni correction is too conservative. You will now apply the Holms method to determining significance of the gene expression test results. In the cell below complete the `holms_significance` function with arguments of the results data frame and the significance level. This function does the following:  
> 1. Find the number of test results and compute the denominator used for the cutoff calculation. 
> 2. Compute the vector of thresholds using the Holms formula. Use the Python `range`function to get the values of the index i. But, keep in mind that range produces a zero-indexed iterator, and the algorithm needs a one-indexed list.  Use the [numpy.divide](https://numpy.org/doc/stable/reference/generated/numpy.divide.html) function to perform the vector divide. Save these threshold values in a data frame in a 'holms_threshold' column.   
> 3. Using the threshold values compute a logical vector and save it in a logical type column named 'significant' in the data frame.
> 4. Return the data frame.
> Finally, execute the function and save the results in a data frame. Then find and print the length of the subset where the 'significance' value is True. 

In [ ]:
def holms_significance(test_results, significance):
    ## First compute the thresholds for each of the ordered tests
    ## Put your code below. 
   




holms_results = holms_significance(gene_statistics, significance_level)    
len(holms_results.loc[holms_results.loc[:,'significant'],:])

> Consider the differences between the calculation of the Bonferroni correction and Holm's threshold. Do you expect any practical difference for large number of test and small test index, $i$? Does this relationship explain the similarity of the results in this case?  
> **End of exercise.**

> **Answer:** 


You can visualize the results of the Holm's method test. The plot has two key elements:  
1. Plot the curve of the p-values vs. the order number, i. The line is color coded by significance or not.
2. Plot the threshold line. This line is straight since the threshold is a linear function of i.

In [ ]:
def plot_significance(results, threshold):
    results['number'] = range(len(results))
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.lineplot(x='number',y=threshold, data=results, ax=ax, color='black', linewidth=0.5)
    sns.scatterplot(x='number',y='p_value', hue='significant', data=results, s=3, ax=ax)
    ax.set_title('Significance of gene expression')
    ax.set_xlabel('Gene number')
    ax.set_ylabel('p-value')
    
plot_significance(holms_results.iloc[:150,:].copy(), 'holms_threshold')    

Notice the following about this plot:  
1. The p-value significance line crosses the threshold point at an apparent break point.   
2. The significant p-values are all very small since there are so many tests.
3. The Holm's threshold is does not change very much over the first 150 test results. 

### Benamini-Hochberg FDR Control 

The Benamini-Hochberg FDR control algorithm is another way to control false discoveries. Stat with an ordered set of $n$ p-values, $D = \{ p_{(1)}, p_{(2)}, p_{(3)}, \ldots, p_{(n)} \}$ we define a false discovery rate, $q$:

$$FDR(D) \le q$$

The cutoff threshold for the ith p-value is then:
$$p_{(i)} \le Threshold(D_q) = \frac{q}{n} i$$

> **Exercise 01-9:** In this exercise you will apply the Benamini-Hochberg FDR control algorithm for testing the significance of the gene expressions. The `BH_significance` function is quite similar to the Holm's method function you have already created. Given the large number of genes you must use a low false discovery rate, $q = 0.001$, or 1 out of 1,000. 
> Execute your function, saving the result. Then print the number of significant cases. 

In [ ]:
def BH_significance(test_results, false_discovery_tollerance):
    ## First compute the thresholds for each of the ordered tests
    ## Put your code below. 
    
    
    
    
    

BH_results = BH_significance(gene_statistics, 0.001)    
len(BH_results.loc[BH_results.loc[:,'significant'],:])

> This result differs from the first two FDR control methods you have applied. Given the false discovery parameter of 0.001 do you think this is a reasonable result and why? 

> **Answer:**       

Finally, execute the code in the cell below and examine the resulting plot.

In [ ]:
plot_significance(BH_results.iloc[:150,:].copy(), 'bh_threshold')    

> **Exercise 01-10**: Compare the plots of results of the Benamini-Hochberg FDR control method to those of Holm's method. Why does the Benamini-Hochberg FDR control method give a more dynamic or adaptive result for the case where $i << n$? 

> **Answer:**       

##### Copyright 2021, 2022, 2023, Stephen F. Elston. All rights reserved. 